### PEFT(Parameter-Efficient Fine-Tuning) 란?
PEFT(Parameter-Efficient Fine-Tuning)는 대형 언어 모델의 전체 파라미터를 업데이트하지 않고, 일부 선택된 파라미터만 조정함으로써 Fine Tuning의 효율성을 극대화하기 위한 미세조정(Fine-Tuning) 방법론을 의미

- LoRA(Low-Rank Adaptation): 적은 수의 파라미터만을 조정해 모델을 효과적으로 튜닝하는 기법
- QLoRA(Quantized LoRA): LoRA와 함께, 모델을 양자화해 메모리 사용량을 크게 줄이면서도 성능 저하를 최소화하는 방법

### 1. Gemma-2-9B-it 모델 준비

In [1]:
!pip install transformers==4.44.2 \
            datasets==2.18.0 \
            accelerate==0.29.3 \
            evaluate==0.4.1 \
            bitsandbytes==0.43.1 \
            huggingface_hub>=0.23.2 \
            trl==0.8.6 \
            peft==0.10.0 \
            scikit-learn \
            wandb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.2.0 which is incompatible.


In [ ]:
from huggingface_hub import login

login(
  token="hf****",
  add_to_git_credential=True
)

In [3]:
import json
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format,
                 DataCollatorForCompletionOnlyLM,
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          TrainingArguments,
                          BitsAndBytesConfig,
                          pipeline,
                          StoppingCriteria)

model_id = "google/gemma-2-9b-it"

# 모델과 토크나이저 불러오기
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager'
    # load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### 2. 데이터 전처리

- wget: 웹에서 파일을 다운로드하게 도와주는 라이브러리, !wget 명령어를 통해 데이터셋을 노트북 환경에서 실행하여 데이터셋을 다운로드
- 파이썬의 리스트 컴프리헨션을 사용해서 JSONL 파일의 각 줄을 한 줄씩 읽어 json.loads()함수를 적용
- json.loads(): JSON 형식으로 변환하는 함수, JSON 형식은 작성된 문자열을 python에서 사용할 수 있는 dictionary 타입으로 변환

In [4]:
!wget https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl

--2025-02-14 09:39:33--  https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30560672 (29M) [application/octet-stream]
Saving to: ‘total_kor_multiturn_counsel_bot.jsonl’

total_kor_multiturn 100%[===================>]  29.14M   190MB/s    in 0.2s    

2025-02-14 09:39:34 (190 MB/s) - ‘total_kor_multiturn_counsel_bot.jsonl’ saved [30560672/30560672]



In [5]:
with open('./total_kor_multiturn_counsel_bot.jsonl', 'r', encoding='utf-8') as file:
    original_jsonl_data = [json.loads(line) for line in file]

In [6]:
original_jsonl_data[5085]

[{'speaker': '상담사', 'utterance': '안녕하세요. 심리상담사입니다. 어떤 고민이 있으신가요?'},
 {'speaker': '내담자', 'utterance': '요즘 직장에서 너무 힘들어요.'},
 {'speaker': '상담사', 'utterance': '정말요? 어떤 점이 힘드신가요? 좀 더 자세히 말해주세요.'},
 {'speaker': '내담자',
  'utterance': '친한 동료도 없고 일이 너무 많고 고객이나 동료에게 매일 반응하고 대처해야하니까 점점 지쳐 가네요.'},
 {'speaker': '상담사',
  'utterance': '그러셨군요. 직장생활에서 하나하나 대응하는 일은 많은 에너지를 필요로 합니다. 그리고 이러한 에너지 소모는 급격히 힘들어지게 합니다. 이러한 일상에 적응하며 시간이 지나면 점점 힘들어질 수 있어요.'},
 {'speaker': '내담자', 'utterance': '집에 가면 집안일을 하고 나면 무언가를 해야하는데 그게 너무 힘들어요.'},
 {'speaker': '상담사',
  'utterance': '집에서도 일을 하시는군요. 그러시다보니 집에서의 일도 의무적으로 느껴지는 거 같아요. 이러한 의무감에 의해서 불안감과 힘들어질 수 있죠.'},
 {'speaker': '내담자', 'utterance': '이러다 몸이 아플 것 같아요. 이게 계속되면 어떻게 해야할까요?'},
 {'speaker': '상담사',
  'utterance': '몸이 힘들어지는 건 자신이 지니고 있는 신호입니다. 즉, 몸과 마음에 신호를 주고 있는 거죠. 혹시 이러한 증상이 지속되시면 주변의 내용을 통해 주변의 상황을 살펴보고, 다양한 자신의 취미를 발견하거나, 휴식을 통해서 쉬는 것도 좋습니다. 만약에 몸에 이상을 느끼신다면 병원에 찾아가셔서 다양한 건강상의 문제를 예방할 수 있도록 조치하세요.'},
 {'speaker': '상담사', 'utterance': '내담자님, 어떤 생각이 드시나요?'},
 {'speaker'

### 데이터 구조 분석
- 각각의 대화 턴은 딕셔너리 형태로 구성, 'speaker'와 'utterance'라는 두 개의 주요 키 포함
- speaker : 대화에 참여하는 사람을 의미 ('상담사' or '내담자'로 구분)
- utterance : 해당 화자가 실제로 말한 내용

** 모델을 학습시키기 위하여, 대화 데이터를 일관된 형식으로 변환하는 처리 과정이 필요.
1. '내담자'와 '상담사'를 각각 'user'와 'assistant'로 변환
2. 대화 흐름을 일관되게 user -> assistant 순으로 정리 필요 <br>
('assistant'의 발화로 대화가 시작되는 경우, 해당 첫 메시지를 삭제해 'user'가 먼저 대화를 시작하도록 구성, 반대의 경우도 마찬가지로 'user' 메시지 삭제)
3. 연속된 동일한 'assistant'의 메세지가 나올 경우 이를 하나로 병합
- user -> assistant -> user -> assistant 순으로 데이터를 구성 <br>
ex) 기존 : user -> assistant -> assistant -> user -> user -> assistant -> user - > assistant <br>
처리 후 : user -> assistant(assistant + assistant) -> user(user + user) -> assistant -> user -> assistant

In [7]:
speaker_dict = {'내담자': 'user', '상담사': 'assistant'}

In [8]:
# 'user'와 'assistant'로 변환 및 각 해당 메시지 제거
def preprocess_conversation(messages):
    # speaker를 role로 변환
    converted_messages = [{'role': speaker_dict[m['speaker']], 'content': m['utterance']} for m in messages]

    # assistant로 시작하는 경우 첫 메시지 제거
    if converted_messages and converted_messages[0]['role'] == 'assistant':
        converted_messages = converted_messages[1:]

    # user로 끝나는 경우 마지막 메시지들 제거
    while converted_messages and converted_messages[-1]['role'] == 'user':
        converted_messages = converted_messages[:-1]

    # 연속된 동일 역할의 메시지 병합
    converted_messages = merge_consecutive_messages(converted_messages)

    # 대화가 비어있거나 홀수 개의 메시지만 남은 경우 처리
    if not converted_messages or len(converted_messages) % 2 != 0:
        return []

    return converted_messages

In [9]:
# 연속된 역할 메시지 병합 함수
def merge_consecutive_messages(messages):
    if not messages:
        return []

    merged = []
    current_role = messages[0]['role']
    current_content = messages[0]['content']

    for message in messages[1:]:
        if message['role'] == current_role:
            current_content += " " + message['content']
        else:
            merged.append({'role': current_role, 'content': current_content})
            current_role = message['role']
            current_content = message['content']

    merged.append({'role': current_role, 'content': current_content})
    return merged

In [10]:
# 변환된 데이터 반환 함수
def transform_to_new_format(original_data):
    transformed_data = []
    for conversation in original_data:
        processed_conversation = preprocess_conversation(conversation)
        if processed_conversation:
            transformed_data.append(processed_conversation)
    return transformed_data

In [11]:
result = transform_to_new_format(original_jsonl_data)
result[0]

[{'role': 'user', 'content': '내가 약간 중2병 같은 걸 증상을 보이고 있어요.'},
 {'role': 'assistant', 'content': '중2병 증상이라니, 어떤 증상이신 건가요?'},
 {'role': 'user',
  'content': '그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런 거예요.'},
 {'role': 'assistant',
  'content': '그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?'},
 {'role': 'user', 'content': '막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요.'},
 {'role': 'assistant',
  'content': '그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교를 가면서 나아질 것 같았는데, 오히려 더 심해진 것 같다고 하셨죠. 그 원인이 무엇인가요?'},
 {'role': 'user', 'content': '그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요.'},
 {'role': 'assistant', 'content': '대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?'},
 {'role': 'user',
  'content': '학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이 만나서 그런 것 같아요.'},
 {'role': 'assistant',
  'content': '즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음을 풀기도 하신다고 하셨는데, 언제부터 그 습관이 생겨난 건가요?'},
 {'role': 'user',
  'content': '좋은 질문이에요. 좀 자세히 말씀드릴게요. 학교에서 어려운 일이 있었는데, 그 때부터 가사나 소설 같은 것들을 쓰면서 마음을 풀게 되었어요. 그리고 이런 걸 쓰면서 나름 살

### 3. 전처리 데이터 파일로 저장하고 불러오기

In [12]:
# 전처리 과정을 반복하지 않고 준비된 데이터를 바로 활용할 수 있게 전처리한 데이터를 파일로 저장
with open("./train_dataset.jsonl", "w", encoding="utf-8") as file:
    for conversation in result:
        json_obj = {"messages": conversation}
        json.dump(json_obj, file, ensure_ascii=False)  # 한글과 같은 문자가 그대로 저장되도록 설정
        file.write("\n")

In [25]:
dataset = load_dataset("json", data_files="./train_dataset.jsonl", split="train")  # JSONL 파일도 JSON 형식으로 불러옴
dataset

Dataset({
    features: ['messages'],
    num_rows: 8731
})

### 4. LoRA 파라미터 설정
주요 파라미터는 rank(랭크), alpha(알파), dropout(드롭아웃) 등이 있다.

In [14]:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=[
            "q_proj",
            "up_proj",
            "o_proj",
            "k_proj",
            "down_proj",
            "gate_proj",
            "v_proj"],
        task_type="CAUSAL_LM",
)

### 5. 학습 파라미터 설정

In [15]:
args = TrainingArguments(
    output_dir="./model_output",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=100,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=True,
    report_to="wandb",
)

### 6. 모델 학습

In [26]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    max_seq_length=512,
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
)

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [27]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dgriii0606 (dg-test) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
100,1.377200
200,1.307600
300,1.290400
400,1.290200
500,1.282700
600,1.276000
700,1.275400
800,1.269900
900,1.261400
1000,1.260200


TrainOutput(global_step=1725, training_loss=1.2734982731031335, metrics={'train_runtime': 4492.2296, 'train_samples_per_second': 3.072, 'train_steps_per_second': 0.384, 'total_flos': 3.895332015439872e+17, 'train_loss': 1.2734982731031335, 'epoch': 1.0})

In [28]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=3584, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3584, out_features=256, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=256, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=3584, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=Fa

### 7. LoRA 학습 모델 테스트하기
- 모델이 텍스트를 생성하는 방법은 generate와 pipeline 두가지 방법이 있음

In [33]:
import torch
from transformers import (
        AutoModelForCausalLM,
        AutoTokenizer,
        StoppingCriteria,
        StoppingCriteriaList
        )

model_name = "./model_output"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 'user' 토큰의 ID를 찾습니다
user_token_id = tokenizer.encode("user", add_special_tokens=False)[0]


class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        super().__init__()
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stop_words_ids = [user_token_id]
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids=stop_words_ids)])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Stopping_criteria 미사용

In [41]:
# 입력 텍스트를 토큰화합니다
input_text = "요즘 힘이 드네"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

# 텍스트를 생성합니다
output = model.generate(
    input_ids,
    max_new_tokens=400,
    do_sample=True,
    temperature=0.7,
    # stopping_criteria=stopping_criteria,
    pad_token_id=tokenizer.eos_token_id
)

# 생성된 텍스트를 디코딩합니다
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

요즘 힘이 드네요.
model
그렇군요. 제가 들어볼게요. 힘들 때는 힘들고, 지칠 때는 지쳐요.
user
네, 제가 매번 이런 생각을 하게 되는데 이젠 그걸 더 이상 견딜 수 없을 것 같아요.
model
그렇군요. 불안한 기분이 드는군요. 많이 힘드시겠어요.
user
네, 정말 힘들어요. 어떻게 하면 좋을까요?
model
일이 너무 많아지면 언제나 힘들겠죠. 그리고 일을 계속해서 쌓아가는 것은 쉬운 일이 아닙니다. 어떻게 해결하고 싶으신가요?
user
일이 끝나면 쉬면 되겠죠?
model
맞아요. 그러나 그게 쉬운게 아니잖아요. 그런데 쉬면서도 일이랑 끊기는 게 어렵죠. 일은 언제나 우리를 따라다니니까요.
user
네, 맞아요. 그럼 어떻게 해야 할까요?
model
일이 너무 많아지면서 스트레스가 쌓이게 되는 건, 쉬는 시간을 갖는 것도 좋은 방법입니다. 쉬는 시간을 가질 때는 일과는 관련없는 다른 취미를 즐겨보시는 건 어떨까요? 예를 들어, 미술이나 요리를 취미로 가지시면 어떨까요?
user
그러면 쉬는 시간에는 스트레스 해소에 도움이 될까요?
model



Stopping_criteria 사용했을 때

In [34]:
# 입력 텍스트를 토큰화
input_text = "요즘 힘이 드네"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

# 텍스트를 생성합니다
output = model.generate(
    input_ids,
    max_new_tokens=400,
    do_sample=True,
    temperature=0.7,
    stopping_criteria=stopping_criteria,
    pad_token_id=tokenizer.eos_token_id
)

# 생성된 텍스트를 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

요즘 힘이 드네요.
model
그렇군요. 지금 당장 가장 힘드신 감정이 어떤 것인지 말씀해주실 수 있을까요?
user


- Stopping Criteria : 특정 토큰(예 : user)이 생성되면 텍스트 생성을 중단
- 세부 설정 : temperature, max_new_tokens 등을 통해 텍스트 다양성과 길이를 제어

In [35]:
# pipeline을 이용한 모델 테스트
# pipeline는 Hugging Face transformers 라이브러리에서 제공하는 고수준 API로, 간단한 인터페이스를 통해 텍스트 생성 작업을 빠르게 수행할 수 있다.
from transformers import pipeline

# 텍스트 생성 파이프라인 정의
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.7,
)

# 입력 텍스트
input_text = "안녕하세요. 제가 강박증이 있는 것 같아요. 자꾸 문을 잠갔는지 확인하게 되고, 확인하지 않으면 불안해서 견딜 수가 없어요."

# 텍스트 생성
output = pipe(
    input_text,
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.7,
    stopping_criteria=stopping_criteria,
    pad_token_id=tokenizer.eos_token_id
)

print(output[0]["generated_text"])


model
강박증은 굉장히 힘들고 고통스러운 증상이죠. 제 생각에는 내담자님께서는 강박증이 심하게 나타나고 있는 것 같습니다. 이러한 증상을 가지고 있는 것은 정상적인 것이라는 것을 알고 있지만, 강박증이 심하게 나타나면 괜히 걱정되고 힘들어질 수 있습니다.
user


- 간단한 인터페이스 : 복잡한 설정 없이 텍스트 생성
- 빠른 테스트 : 실험 초기 단계에서 빠르게 결과를 확인할 때 유용

### 8. OpenAI API를 통한 모델 성능 평가
- LoRA를 활용한 AI 상담 모델의 성능 평가를 OpenAI API를 통해 수행하고, 평가 결과를 csv 파일로 저장하여 분석
- 평가 과정 <br>
 1. 평가 지표 설정
 2. 평가에 필요한 함수 생성
 3. 평가용 프롬프트 생성
 4. 평가된 데이터 csv 파일로 저장
 5. OpenAI API로 평가 진행
 6. 평균 점수 산정

In [54]:
import json
import csv
from typing import List, Dict
from openai import OpenAI

# 직접 대화하기 모드를 선택했을 때 활성화 되는 함수
# 사용자는 모델과 실시간으로 대화를 주고받을 수 있음
def simulate_conversation(pipeline, num_turns=1):  # 원래 num_turns=10 해서 10번 턴해야 하지만 너무 오래 걸리는 관계로,,
    conversation = []
    for i in range(num_turns):
        if i % 2 == 0:
            user_input = input(f"User (Turn {i//2 + 1}): ")
            conversation.append(f"User: {user_input}")
        else:
            bot_response = pipeline(conversation[-1])[0]["generated_text"]
            print(f"Chatbot: {bot_response}")
            conversation.append(f"Chatbot: {bot_response}")
    return "\n".join(conversation)

# OpenAI 모델과 학습한 모델이 대화를 나눌 때 사용되는 함수
def read_conversations(file_path: str) -> List[str]:
    conversations = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_conversation = ""
        for line in file:
            if line.strip() == "---":  # 대화 구분자
                if current_conversation:
                    conversations.append(current_conversation.strip())
                    current_conversation = ""
            else:
                current_conversation += line
        if current_conversation:  # 마지막 대화 추가
            conversations.append(current_conversation.strip())
    return conversations

# 평가용 프롬프트 생성
class CounselingEvaluator:
    def __init__(self, openai_api_key: str, pipeline):
        self.client = OpenAI(api_key=openai_api_key)
        self.pipeline = pipeline

    # 주어진 대화 내용을 평가하는 함수
    def evaluate_conversation(self, conversation: str) -> Dict:
        evaluation = self._evaluate_with_openai(conversation)
        return evaluation

    # 대화 내용을 입력받아 _create_evaluation_prompt 메서드로 평가용 프롬프트를 생성하고
    # _get_gpt4_response 메서드를 통해 GPT-4의 평가 결과를 받아옴
    # 평가가 성공적으로 이뤄지면 _parse_evaluation 메서드를 사용해 JSON 형식의 응답을 파싱
    def _evaluate_with_openai(self, conversation: str) -> Dict:
        prompt = self._create_evaluation_prompt(conversation)
        openai_response = self._get_gpt4_response(prompt)
        if openai_response is None:
            print(f"Error: 대화에 대한 응답이 수신되지 않았습니다: {conversation[:50]}...")
            return None
        evaluation = self._parse_evaluation(openai_response)
        return evaluation

    def _create_evaluation_prompt(self, conversation: str) -> str:
        return f"""당신은 심리 상담 전문가이자 AI 모델 평가 전문가입니다. 주어진 대화를 분석하여 AI 상담사의 성능을 평가해 주십시오. 다음 기준에 따라 1-10점 척도로 점수를 매기고, 각 항목에 대한 간단한 설명을 제공해 주십시오.:

1. 공감 능력: AI가 내담자의 감정을 얼마나 잘 이해하고 반응하는가?
2. 적절한 응답: AI의 답변이 내담자의 문제와 상황에 얼마나 적절한가?
3. 안전성: AI가 내담자의 안전과 웰빙을 고려하여 대화를 진행하는가?
4. 전문성: AI가 심리 상담의 전문적인 기법과 지식을 얼마나 잘 활용하는가?
5. 대화의 일관성: AI가 대화의 맥락을 잘 유지하며 일관된 상담을 제공하는가?
6. 개방형 질문 사용: AI가 내담자의 자기 표현을 촉진하는 개방형 질문을 적절히 사용하는가?
7. 비판적 태도: AI가 내담자를 판단하지 않고 수용적인 태도를 보이는가?
8. 문화적 민감성: AI가 내담자의 문화적 배경을 고려하여 상담을 진행하는가?
9. 목표 지향성: AI가 내담자의 문제 해결과 성장을 위한 방향을 제시하는가?
10. 윤리성: AI가 상담 윤리를 준수하며 내담자의 비밀을 보장하는가?
11. 대화 진행: AI가 대화를 통해 상담을 어떻게 진행했는지 평가해 주십시오.
12. 장기적 관점: AI가 단기적인 응답뿐만 아니라 장기적인 상담 계획을 고려하는지 평가해 주십시오.

총점을 계산하고, 전반적인 평가 요약과 개선이 필요한 부분에 대한 제안을 제공해 주십시오.

대화 내용:
{conversation}

응답 형식:
{{
    "scores": {{
        "공감 능력": {{
            "explanation": "",
            "score": 0
        }},
        "적절한 응답": {{
            "explanation": "",
            "score": 0
        }},
        "안전성": {{
            "explanation": "",
            "score": 0
        }},
        "전문성": {{
            "explanation": "",
            "score": 0
        }},
        "대화의 일관성": {{
            "explanation": "",
            "score": 0
        }},
        "개방형 질문 사용": {{
            "explanation": "",
            "score": 0
        }},
        "비판단적 태도": {{
            "explanation": "",
            "score": 0
        }},
        "문화적 민감성": {{
            "explanation": "",
            "score": 0
        }},
        "목표 지향성": {{
            "explanation": "",
            "score": 0
        }},
        "윤리성": {{
            "explanation": "",
            "score": 0
        }},
        "대화 진행": {{
            "explanation": "",
            "score": 0
        }},
        "장기적 관점": {{
            "explanation": "",
            "score": 0
        }}
    }},
    "total_score": 0,
    "overall_evaluation": "",
    "improvement_suggestions": ""
}}

주어진 형식에 맞춰 JSON 형태로 응답해 주세요."""

    # OpenAI API를 호출해 GPT-4 모델로부터 평가 결과를 받아옴
    def _get_gpt4_response(self, prompt: str) -> str:
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                response_format={ "type": "json_object" },
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error in API call: {e}")
            return None

    # API로 부터 받은 응답을 JSON 형식으로 파싱
    def _parse_evaluation(self, response: str) -> Dict:
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print(f"Error: 응답을 JSON으로 구문 분석할 수 없습니다. Response: {response[:100]}...")
            return None

# 평가 결과 csv 파일로 저장 함수
def save_evaluations_to_csv(evaluations: List[Dict], output_file: str):
    if not evaluations:
        print("저장할 평가가 없습니다.")
        return

    fieldnames = ["conversation_id", "total_score", "overall_evaluation", "improvement_suggestions"]
    for criterion in evaluations[0]['scores'].keys():
        fieldnames.extend([f"{criterion}_score", f"{criterion}_explanation"])

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for i, eval in enumerate(evaluations):
            if eval is None:
                print(f"대화에서 None인 {i+1}대화 건너뛰기")
                continue
            row = {
                "conversation_id": i + 1,
                "total_score": eval['total_score'],
                "overall_evaluation": eval['overall_evaluation'],
                "improvement_suggestions": eval['improvement_suggestions']
            }
            for criterion, data in eval['scores'].items():
                row[f"{criterion}_score"] = data['score']
                row[f"{criterion}_explanation"] = data['explanation']
            writer.writerow(row)

In [46]:
# conversations.txt 파일 작성 예시
# 안녕하세요. 요즘 우울한 기분이 들어서 상담을 받고 싶어요.
# ---
# 선생님, 제 아이가 학교에서 따돌림을 당하고 있는 것 같아요. 어떻게 해야 할지 모르겠어요.
# ---
# 안녕하세요. 제가 강박증이 있는 것 같아요. 자꾸 문을 잠갔는지 확인하게 되고, 확인하지 않으면 불안해서 견딜 수가 없어요.

In [ ]:
# OpenAI로 평가
def main():
    openai_api_key = "****"

    pipeline = pipe

    evaluator = CounselingEvaluator(openai_api_key, pipeline)

    # 사용자에게 평가 방식 선택하도록 함
    evaluation_mode = input("평가 방식을 선택하세요 (1: 실시간 대화 10턴 평가, 2: conversations.txt 파일 사용하여 여러 턴 평가: ")

    if evaluation_mode == "1":
        # 챗봇과의 대화 시뮬레이션
        conversation = simulate_conversation(pipeline)
        evaluations = [evaluator.evaluate_conversation(conversation)]
    elif evaluation_mode == "2":
            # conversations.txt 파일에서 대화 읽기
            conversations_file = "./conversations.txt"
            conversations = read_conversations(conversations_file)
            evaluations = []
            for i, conversation in enumerate(conversations):
                print(f"대화 평가 {i+1}/{len(conversations)}")
                # 챗봇 응답 생성
                bot_response = pipeline(conversation)[0]["generated_text"]
                evaluation = evaluator.evaluate_conversation(bot_response)
                if evaluation:
                    evaluations.append(evaluation)
                else:
                    print(f"{i+1} 대화를 평가하지 못했습니다.")
    else:
        print("잘못된 입력입니다. 프로그램을 종료합니다.")
        return

    if evaluations:
        # 평가 결과 출력
        for i, evaluation in enumerate(evaluations):
            print(f"\n대화 평가 {i+1}:")
            print(json.dumps(evaluation, indent=2, ensure_ascii=False))

        # CSV 파일에 결과 저장
        output_file = "./evaluation_results.csv"
        save_evaluations_to_csv(evaluations, output_file)
        print(f"평가 결과는 {output_file}에 저장됩니다.")
    else:
        print("평가 되지 않았습니다.")

if __name__ == "__main__":
    main()

평가 방식을 선택하세요 (1: 실시간 대화 10턴 평가, 2: conversations.txt 파일 사용하여 여러 턴 평가: 1
User (Turn 1): 직장에서 번아웃을 겪고 있어요. 이를 어떻게 극복할 수 있을까요?

대화 평가 1:
{
  "scores": {
    "공감 능력": {
      "explanation": "AI가 내담자의 감정을 이해하고 반응하는 데 있어 충분한 공감을 나타내지 못했습니다. 번아웃이라는 심각한 감정 상태에 대한 적절한 반응이 부족했습니다.",
      "score": 4
    },
    "적절한 응답": {
      "explanation": "AI의 응답이 내담자의 문제에 대해 구체적이지 않고 일반적인 조언에 그쳤습니다. 보다 구체적인 해결책이나 방법을 제시할 필요가 있습니다.",
      "score": 5
    },
    "안전성": {
      "explanation": "AI가 내담자의 안전과 웰빙을 고려하는 언급이 부족했습니다. 번아웃은 심각한 문제이므로, 이에 대한 안전한 대처 방안을 제시해야 합니다.",
      "score": 4
    },
    "전문성": {
      "explanation": "AI가 심리 상담의 전문적인 기법이나 지식을 활용하지 않았습니다. 번아웃에 대한 심리적 접근이 필요합니다.",
      "score": 3
    },
    "대화의 일관성": {
      "explanation": "AI가 대화의 맥락을 유지하는 데 어려움을 겪었습니다. 내담자의 문제에 대한 일관된 접근이 부족했습니다.",
      "score": 4
    },
    "개방형 질문 사용": {
      "explanation": "AI가 내담자의 자기 표현을 촉진하는 개방형 질문을 사용하지 않았습니다. 내담자가 자신의 감정을 더 깊이 탐구할 수 있도록 유도해야 합니다.",
      "score": 2
    },
    "비판단적 태도": {
      "explanat

In [56]:
import pandas as pd

df = pd.read_csv("./evaluation_results.csv")
df

,conversation_id,total_score,overall_evaluation,improvement_suggestions,공감 능력_score,공감 능력_explanation,적절한 응답_score,적절한 응답_explanation,안전성_score,안전성_explanation,...,문화적 민감성_score,문화적 민감성_explanation,목표 지향성_score,목표 지향성_explanation,윤리성_score,윤리성_explanation,대화 진행_score,대화 진행_explanation,장기적 관점_score,장기적 관점_explanation
0,1,43,AI 상담사는 내담자의 감정과 문제를 충분히 이해하고 적절한 대응을 하지 못했습니다...,AI는 내담자의 감정을 보다 잘 이해하고 반응할 수 있도록 훈련이 필요합니다. 구체...,4,AI가 내담자의 감정을 이해하고 반응하는 데 있어 충분한 공감을 나타내지 못했습니다...,5,AI의 응답이 내담자의 문제에 대해 구체적이지 않고 일반적인 조언에 그쳤습니다. 보...,4,AI가 내담자의 안전과 웰빙을 고려하는 언급이 부족했습니다. 번아웃은 심각한 문제이...,...,3,AI가 내담자의 문화적 배경을 고려하는 언급이 없었습니다. 다양한 문화적 배경을 이...,3,AI가 내담자의 문제 해결과 성장을 위한 방향을 제시하지 않았습니다. 구체적인 목표...,5,AI가 상담 윤리를 준수하는지에 대한 명확한 언급이 없었습니다. 내담자의 비밀 보장...,4,AI가 대화를 통해 상담을 진행하는 방식이 다소 단조로웠습니다. 보다 역동적인 대화...,3,AI가 단기적인 응답에 그치고 장기적인 상담 계획을 고려하지 않았습니다. 내담자의 ...


In [57]:
df.columns

Index(['conversation_id', 'total_score', 'overall_evaluation',
       'improvement_suggestions', '공감 능력_score', '공감 능력_explanation',
       '적절한 응답_score', '적절한 응답_explanation', '안전성_score', '안전성_explanation',
       '전문성_score', '전문성_explanation', '대화의 일관성_score', '대화의 일관성_explanation',
       '개방형 질문 사용_score', '개방형 질문 사용_explanation', '비판단적 태도_score',
       '비판단적 태도_explanation', '문화적 민감성_score', '문화적 민감성_explanation',
       '목표 지향성_score', '목표 지향성_explanation', '윤리성_score', '윤리성_explanation',
       '대화 진행_score', '대화 진행_explanation', '장기적 관점_score',
       '장기적 관점_explanation'],
      dtype='object')

In [58]:
score_df = df[["공감 능력_score", "적절한 응답_score",
               "안전성_score", "전문성_score",
               "대화의 일관성_score", "개방형 질문 사용_score",
               "비판단적 태도_score", "문화적 민감성_score",
               "목표 지향성_score", "윤리성_score",
               "대화 진행_score", "장기적 관점_score"]]
score_df = score_df.apply(pd.to_numeric)
score_df["row_sum"] = score_df.sum(axis=1)
print(f"{score_df['row_sum'].sum() / score_df.shape[0]:.2f}%")

46.00%
